In [2]:
"""
Created on Wed Jul 19 05:16:49 2023
@author: hugo511
"""

import argparse
import pandas as pd
# pd.set_option('display.float_format', lambda x:'%.6f'%x)
# pd.reset_option('display.float_format')
import numpy as np
np.set_printoptions(suppress=True)
import time
import warnings
warnings.filterwarnings('ignore')
import os 
import statsmodels.api as sm
from numpy import mat
import datetime
from dateutil.relativedelta import relativedelta

parser = argparse.ArgumentParser(description="Asset Pricing - Tail stocks return regression")
parser.add_argument('--dataset_start', type=int, default=196301, help='dataset start date')
parser.add_argument('--dataset_end', type=int, default=201012, help='dataset end date')
parser.add_argument('--qt', type=float, default=0.05, help='quantile value - u_t')
parser.add_argument('--save_dir', type=str, default='../testTailStocksRET_table4/', help='results save dirs')
parser.add_argument('--isTest', type=bool, default=True, help='is Test or not')
parser.add_argument('--isExcludeBelow', type=bool, default=False, help='exclude stocks below 5 at portfolio formation date')
parser.add_argument('--H', type=int, default=1, help='portfolio holding period return')
parser.add_argument('--Lwindow', type=int, default=1, help='length of rolling window')
parser.add_argument('--loading', type=str, default='init', help='loading types, choices: init, predictive')
parser.add_argument('-f')
args = parser.parse_args()

if __name__ == '__main__':
    # dataset
    args.dataset_start = 196301
    args.dataset_end = 202212 
    args.H = 1
    args.loading = 'predictive'
    args.isTest = False 
    args.tail_dir = './logs/Hill_estimate_results/'
    args.isExcludeBelow = True
    # tail lambda_t
    tail_estimates = pd.read_csv(os.path.join(args.tail_dir, f'Hillestimate_plotresults_{args.dataset_start}_{args.dataset_end}.csv'))
    tail_estimates = tail_estimates[(tail_estimates['date'] >= args.dataset_start) & (tail_estimates['date'] <= args.dataset_end)]
    tail_estimates.rename(columns={'lambda_t':'Tail'}, inplace=True)
    
    # allstocks bycolumn Monthly 
    allstocks_ret = pd.read_csv('../dataset20230614/allstocks_ret_bycolumn_Monthly_192601_202212.csv')
    allstocks_ret.columns
    allstocks_ret_dataset = allstocks_ret[((allstocks_ret['SHRCD'] == 10.) | (allstocks_ret['SHRCD'] == 11.)) \
                                          & ((allstocks_ret['EXCHCD'] == 1) | (allstocks_ret['EXCHCD'] == 2) | (allstocks_ret['EXCHCD'] == 3))]
    allstocks_ret_dataset = allstocks_ret_dataset[['PERMNO', 'date', 'RET', 'PRC', 'SHROUT']]
    allstocks_ret_dataset['date'] = allstocks_ret_dataset['date'].apply(lambda x:int(str(x)[:6]))
    allstocks_ret_dataset = allstocks_ret_dataset[(allstocks_ret_dataset['date'] >= args.dataset_start) & (allstocks_ret_dataset['date'] <= args.dataset_end)]
    allstocks_ret_dataset[(allstocks_ret_dataset['RET'] == 'B') | (allstocks_ret_dataset['RET'] == 'C')] = np.nan
    allstocks_ret_dataset['RET'] = allstocks_ret_dataset['RET'].astype(float)
    allstocks_ret_dataset['CAP'] = allstocks_ret_dataset['PRC'] * allstocks_ret_dataset['SHROUT']

    date_lt = tail_estimates['date'].unique().tolist()


In [3]:
#%% # beta loading roop 
labelc = ['Low', '2', '3', '4', 'High']
date_lt.sort()
betaloading_dates = {}
sorted_portfolios_ewretY = pd.DataFrame(columns=['date'] + labelc)
sorted_portfolios_vwretY = pd.DataFrame(columns=['date'] + labelc)
for idx in range(date_lt.index(197212), len(date_lt) - args.H, args.H):
    print(idx, date_lt[idx])        
    time1 = time.time()
    formation_date = date_lt[idx]
    # snapshot dataset 
    snapshot_start = date_lt[idx - 120 + 1]
    snapshot_end = date_lt[idx]
    snapshot_dataset = allstocks_ret_dataset[(allstocks_ret_dataset['date'] >= snapshot_start) & (allstocks_ret_dataset['date'] <= snapshot_end)].drop_duplicates().reset_index(drop=True)
    # exclude stock price < 5. at portfolio formation date 
    include_stocks_PRC = snapshot_dataset[(snapshot_dataset['date'] == formation_date) & (snapshot_dataset['PRC'] >= 5)]['PERMNO'].unique().tolist()
    # just include stocks with >= 36M RET in 120M
    snapshot_dataset['RET_cnt'] = snapshot_dataset['RET'].apply(lambda x : 0. if x=='B' or x=='C' or np.isnan(float(x)) else 1.)
    snapshot_stat = snapshot_dataset.groupby(by=['PERMNO']).agg({'RET_cnt':'sum'}).reset_index()
    include_stocks_RETcnt = snapshot_stat[snapshot_stat['RET_cnt'] >= 36]['PERMNO'].unique().tolist()
    # stocks pool
    include_stocks = list(set(include_stocks_PRC) & set(include_stocks_RETcnt))

    # Estimate stocks tail loading 
    betaloading_dataset = pd.DataFrame(include_stocks, columns=['PERMNO']).merge(snapshot_dataset[['PERMNO', 'date', 'RET']], how='left', on='PERMNO')
    betaloading_dataset = betaloading_dataset.merge(tail_estimates[['date', 'Tail']], how='left', on='date')
    betaloading_stocks = {}
    for s in include_stocks:
        # print(s)
        if args.loading == 'init':
            fitY = betaloading_dataset[betaloading_dataset['PERMNO'] == s]['RET']
            fitX = betaloading_dataset[betaloading_dataset['PERMNO'] == s]['Tail']
            fitX = sm.add_constant(fitX)            
            rgmodel = sm.OLS(fitY, fitX).fit()
            tailloading = rgmodel.params['Tail']
            betaloading_stocks[s] = tailloading
        elif args.loading == 'predictive':
            fitXY = betaloading_dataset[betaloading_dataset['PERMNO'] == s][['date', 'Tail', 'RET']]
            fitXY['RET_t+1'] = fitXY['RET'].shift(-1)
            fitXY = fitXY.dropna(axis=0)
            fitY = fitXY['RET_t+1']
            fitX = fitXY['Tail']
            fitX = sm.add_constant(fitX)
            rgmodel = sm.OLS(fitY, fitX).fit()  
            tailloading = rgmodel.params['Tail']
            betaloading_stocks[s] = tailloading
        elif args.loading == 'difference':
            fitXY = betaloading_dataset[betaloading_dataset['PERMNO'] == s][['date', 'Tail', 'RET']]
            fitXY['RET_t+1'] = fitXY['RET'].shift(-1)
            fitXY['Tail_t-1'] = fitXY['Tail'].shift(1)
            fitXY['Tail_diff'] = fitXY['Tail'] - fitXY['Tail_t-1']
            fitXY = fitXY.dropna(axis=0)
            fitY = fitXY['RET_t+1']
            fitX = fitXY['Tail_diff'] 
            fitX = sm.add_constant(fitX)
            rgmodel = sm.OLS(fitY, fitX).fit()
            tailloading = rgmodel.params['Tail_diff']
            betaloading_stocks[s] = tailloading
    betaloading_dates[formation_date] = betaloading_stocks
    betaloading_df = pd.DataFrame(betaloading_stocks, index=[0]).T.reset_index()
    betaloading_df.columns = ['PERMNO', 'Tailbeta']

    # sort and form portfolios
    sorted_portfolios = betaloading_df
    sorted_portfolios['groups'] = pd.qcut(sorted_portfolios['Tailbeta'], q=5, labels=['Low', '2', '3', '4', 'High'])

    # holding period dataset and value-weighted weigth
    holding_start = date_lt[idx + 1]
    holding_end = date_lt[idx + args.H]
    holding_dataset = allstocks_ret_dataset[(allstocks_ret_dataset['date'] >= holding_start) & (allstocks_ret_dataset['date'] <= holding_end)][['PERMNO', 'date', 'RET']].drop_duplicates().reset_index(drop=True)
    # cap weight - CAP_{t} for ret_{t+1}
    weight_start = date_lt[idx]
    weight_end = date_lt[idx + args.H - 1]            
    weight_dataset = allstocks_ret_dataset[(allstocks_ret_dataset['date'] >= weight_start) & (allstocks_ret_dataset['date'] <= weight_end)][['PERMNO', 'date', 'CAP']].drop_duplicates().reset_index(drop=True)
    weight_dataset = sorted_portfolios[['PERMNO', 'groups']].merge(weight_dataset, how='left', on='PERMNO')
    cap_sum = weight_dataset.groupby(by=['groups', 'date']).agg({'CAP':'sum'}).reset_index()
    cap_sum.rename(columns={'CAP':'CAPSUM'}, inplace=True)
    weight_dataset = weight_dataset.merge(cap_sum, how='left', on=['groups', 'date'])
    weight_dataset['date'] = weight_dataset['date'].apply(lambda x: int(datetime.datetime.strftime(datetime.datetime.strptime(str(int(x)), '%Y%m') + relativedelta(months=1), '%Y%m')))
    # holding period return for every stocks in portfolios
    holding_period_returns = weight_dataset.merge(holding_dataset, how='left', on=['PERMNO', 'date'])
    holding_period_returns['RET_vw'] = holding_period_returns['RET'] * holding_period_returns['CAP'] / holding_period_returns['CAPSUM']

    # calculate portfortlio returns - equal weighted / value weighted
    # equal-weighted
    sorted_returns_ewret = holding_period_returns.groupby(by=['groups', 'date']).agg({'RET':'mean'}).reset_index()
    sorted_returns_ewrety = pd.DataFrame()
    sorted_returns_ewrety['date'] = sorted_returns_ewret['date'].unique()
    for g in labelc:
        sorted_returns_ewrety = pd.concat([sorted_returns_ewrety, sorted_returns_ewret[sorted_returns_ewret['groups'] == g]['RET'].reset_index(drop=True)], axis=1)
    sorted_returns_ewrety.columns = ['date'] + labelc
    sorted_portfolios_ewretY = pd.concat([sorted_portfolios_ewretY, sorted_returns_ewrety], axis=0)
    # value-weighted
    sorted_returns_vwret = holding_period_returns.groupby(by=['groups', 'date']).agg({'RET_vw':'sum'}).reset_index()
    sorted_returns_vwrety = pd.DataFrame()        
    sorted_returns_vwrety['date'] = sorted_returns_vwret['date'].unique()      
    for g in labelc:
        sorted_returns_vwrety = pd.concat([sorted_returns_vwrety, sorted_returns_vwret[sorted_returns_vwret['groups'] == g]['RET_vw'].reset_index(drop=True)], axis=1)
    sorted_returns_vwrety.columns = ['date'] + labelc
    sorted_portfolios_vwretY = pd.concat([sorted_portfolios_vwretY, sorted_returns_vwrety], axis=0)

    time2 = time.time()
    print(f'============ finish {formation_date}. {(time2 - time1) / 60:.2f} mins. ============')
args.save_dir = './logs/PortfolioSort/'
if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir)


119 197212.0
============ finish 197212.0. 0.13 mins. ============
120 197301.0
============ finish 197301.0. 0.12 mins. ============
121 197302.0
============ finish 197302.0. 0.12 mins. ============
122 197303.0
============ finish 197303.0. 0.13 mins. ============
123 197304.0
============ finish 197304.0. 0.13 mins. ============
124 197305.0
============ finish 197305.0. 0.12 mins. ============
125 197306.0
============ finish 197306.0. 0.12 mins. ============
126 197307.0
============ finish 197307.0. 0.12 mins. ============
127 197308.0
============ finish 197308.0. 0.12 mins. ============
128 197309.0
============ finish 197309.0. 0.13 mins. ============
129 197310.0
============ finish 197310.0. 0.13 mins. ============
130 197311.0
============ finish 197311.0. 0.12 mins. ============
131 197312.0
============ finish 197312.0. 0.12 mins. ============
132 197401.0
============ finish 197401.0. 0.12 mins. ============
133 197402.0
============ finish 197402.0. 0.12 mins. ========

============ finish 198303.0. 0.15 mins. ============
243 198304.0
============ finish 198304.0. 0.15 mins. ============
244 198305.0
============ finish 198305.0. 0.15 mins. ============
245 198306.0
============ finish 198306.0. 0.16 mins. ============
246 198307.0
============ finish 198307.0. 0.16 mins. ============
247 198308.0
============ finish 198308.0. 0.17 mins. ============
248 198309.0
============ finish 198309.0. 0.16 mins. ============
249 198310.0
============ finish 198310.0. 0.16 mins. ============
250 198311.0
============ finish 198311.0. 0.16 mins. ============
251 198312.0
============ finish 198312.0. 0.17 mins. ============
252 198401.0
============ finish 198401.0. 0.17 mins. ============
253 198402.0
============ finish 198402.0. 0.17 mins. ============
254 198403.0
============ finish 198403.0. 0.17 mins. ============
255 198404.0
============ finish 198404.0. 0.17 mins. ============
256 198405.0
============ finish 198405.0. 0.17 mins. ============
257 1984

============ finish 199306.0. 0.25 mins. ============
366 199307.0
============ finish 199307.0. 0.25 mins. ============
367 199308.0
============ finish 199308.0. 0.31 mins. ============
368 199309.0
============ finish 199309.0. 0.26 mins. ============
369 199310.0
============ finish 199310.0. 0.24 mins. ============
370 199311.0
============ finish 199311.0. 0.22 mins. ============
371 199312.0
============ finish 199312.0. 0.23 mins. ============
372 199401.0
============ finish 199401.0. 0.24 mins. ============
373 199402.0
============ finish 199402.0. 0.23 mins. ============
374 199403.0
============ finish 199403.0. 0.23 mins. ============
375 199404.0
============ finish 199404.0. 0.23 mins. ============
376 199405.0
============ finish 199405.0. 0.22 mins. ============
377 199406.0
============ finish 199406.0. 0.23 mins. ============
378 199407.0
============ finish 199407.0. 0.22 mins. ============
379 199408.0
============ finish 199408.0. 0.23 mins. ============
380 1994

============ finish 200309.0. 0.25 mins. ============
489 200310.0
============ finish 200310.0. 0.26 mins. ============
490 200311.0
============ finish 200311.0. 0.25 mins. ============
491 200312.0
============ finish 200312.0. 0.26 mins. ============
492 200401.0
============ finish 200401.0. 0.26 mins. ============
493 200402.0
============ finish 200402.0. 0.26 mins. ============
494 200403.0
============ finish 200403.0. 0.25 mins. ============
495 200404.0
============ finish 200404.0. 0.25 mins. ============
496 200405.0
============ finish 200405.0. 0.25 mins. ============
497 200406.0
============ finish 200406.0. 0.26 mins. ============
498 200407.0
============ finish 200407.0. 0.24 mins. ============
499 200408.0
============ finish 200408.0. 0.24 mins. ============
500 200409.0
============ finish 200409.0. 0.25 mins. ============
501 200410.0
============ finish 200410.0. 0.25 mins. ============
502 200411.0
============ finish 200411.0. 0.26 mins. ============
503 2004

============ finish 201312.0. 0.20 mins. ============
612 201401.0
============ finish 201401.0. 0.20 mins. ============
613 201402.0
============ finish 201402.0. 0.20 mins. ============
614 201403.0
============ finish 201403.0. 0.20 mins. ============
615 201404.0
============ finish 201404.0. 0.20 mins. ============
616 201405.0
============ finish 201405.0. 0.20 mins. ============
617 201406.0
============ finish 201406.0. 0.20 mins. ============
618 201407.0
============ finish 201407.0. 0.20 mins. ============
619 201408.0
============ finish 201408.0. 0.19 mins. ============
620 201409.0
============ finish 201409.0. 0.20 mins. ============
621 201410.0
============ finish 201410.0. 0.20 mins. ============
622 201411.0
============ finish 201411.0. 0.19 mins. ============
623 201412.0
============ finish 201412.0. 0.19 mins. ============
624 201501.0
============ finish 201501.0. 0.19 mins. ============
625 201502.0
============ finish 201502.0. 0.20 mins. ============
626 2015

In [5]:
#%% Alphas
equal_weighted_table = pd.DataFrame(sorted_portfolios_ewretY.iloc[:,1:].mean(axis=0)).T * 100
equal_weighted_table.index = ['Equal weighted AvgRET']    
value_weighted_table = pd.DataFrame(sorted_portfolios_vwretY.iloc[:, 1:].mean(axis=0)).T * 100
value_weighted_table.index = ['Value weighted AvgRET']

# Factors dataset 
# FF_dataset 
FF_dataset = pd.read_csv('./Dataset/FF3factors_Momentum_Monthly_192601_202212.csv')
FF_dataset['date'] = FF_dataset['dateff'].apply(lambda x: int(str(x)[:6]))
FF_dataset = FF_dataset[['date', 'mktrf', 'smb', 'hml', 'umd', 'rf']]
# PSLiquid dataset 
PSLiq_dataset = pd.read_csv('./Dataset/PSLiquidityFactors_Monthly_196208_202212.csv') 
PSLiq_dataset['date'] = PSLiq_dataset['DATE'].apply(lambda x:int(str(x)[:6]))
PSLiq_dataset = PSLiq_dataset[['date', 'PS_VWF']]

# Y:portfolio returns monthly
Y_ewret = sorted_portfolios_ewretY.reset_index(drop=True)
Y_vwret = sorted_portfolios_vwretY.reset_index(drop=True)

# CAPM 
fitXY_ewret = Y_ewret.merge(FF_dataset[['date', 'mktrf', 'rf']], how='left', on='date')
fitY_ewret = fitXY_ewret[labelc] - mat(fitXY_ewret['rf']).T @ np.ones([1, 5])  
fitX_ewret = fitXY_ewret['mktrf']    
fitX_ewret = sm.add_constant(fitX_ewret)
rgmodel = sm.OLS(fitY_ewret, fitX_ewret).fit()
CAPM_alphas = pd.DataFrame(rgmodel.params.loc['const']).T * 100
CAPM_alphas.index = ['CAPM_alpha']; CAPM_alphas.columns = labelc
equal_weighted_table = pd.concat([equal_weighted_table, CAPM_alphas], axis=0)

fitXY_vwret = Y_vwret.merge(FF_dataset[['date', 'mktrf', 'rf']], how='left', on='date')
fitY_vwret = fitXY_vwret[labelc] - mat(fitXY_vwret['rf']).T @ np.ones([1, 5])  
fitX_vwret = fitXY_vwret['mktrf']    
fitX_vwret = sm.add_constant(fitX_vwret)
rgmodel = sm.OLS(fitY_vwret, fitX_vwret).fit()
CAPM_alphas = pd.DataFrame(rgmodel.params.loc['const']).T * 100
CAPM_alphas.index = ['CAPM_alpha']; CAPM_alphas.columns = labelc
value_weighted_table = pd.concat([value_weighted_table, CAPM_alphas], axis=0)

# FF3 
fitXY_ewret = Y_ewret.merge(FF_dataset[['date', 'mktrf', 'smb', 'hml', 'rf']], how='left', on='date')
fitY_ewret = fitXY_ewret[labelc] - mat(fitXY_ewret['rf']).T @ np.ones([1, 5])
fitX_ewret = fitXY_ewret[['mktrf', 'smb', 'hml']]
fitX_ewret = sm.add_constant(fitX_ewret)
rgmodel = sm.OLS(fitY_ewret, fitX_ewret).fit()
FF_alphas = pd.DataFrame(rgmodel.params.loc['const']).T * 100
FF_alphas.index = ['FF_alphs']; FF_alphas.columns = labelc
equal_weighted_table = pd.concat([equal_weighted_table, FF_alphas], axis=0)

fitXY_vwret = Y_vwret.merge(FF_dataset[['date', 'mktrf', 'smb', 'hml', 'rf']], how='left', on='date')
fitY_vwret = fitXY_vwret[labelc] - mat(fitXY_vwret['rf']).T @ np.ones([1, 5])
fitX_vwret = fitXY_vwret[['mktrf', 'smb', 'hml']]
fitX_vwret = sm.add_constant(fitX_vwret)
rgmodel = sm.OLS(fitY_vwret, fitX_vwret).fit()
FF_alphas = pd.DataFrame(rgmodel.params.loc['const']).T * 100
FF_alphas.index = ['FF_alphs']; FF_alphas.columns = labelc
value_weighted_table = pd.concat([value_weighted_table, FF_alphas], axis=0)

# FF + Momentum
fitXY_ewret = Y_ewret.merge(FF_dataset[['date', 'mktrf', 'smb', 'hml', 'umd', 'rf']], how='left', on='date')
fitY_ewret = fitXY_ewret[labelc] - mat(fitXY_ewret['rf']).T @ np.ones([1, 5])
fitX_ewret = fitXY_ewret[['mktrf', 'smb', 'hml', 'umd']]
fitX_ewret = sm.add_constant(fitX_ewret)
rgmodel = sm.OLS(fitY_ewret, fitX_ewret).fit()
FFMom_alphas = pd.DataFrame(rgmodel.params.loc['const']).T * 100
FFMom_alphas.index = ['FFMom_alphas']; FFMom_alphas.columns = labelc
equal_weighted_table = pd.concat([equal_weighted_table, FFMom_alphas], axis=0)

fitXY_vwret = Y_vwret.merge(FF_dataset[['date', 'mktrf', 'smb', 'hml', 'umd', 'rf']], how='left', on='date')
fitY_vwret = fitXY_vwret[labelc] - mat(fitXY_vwret['rf']).T @ np.ones([1, 5])
fitX_vwret = fitXY_vwret[['mktrf', 'smb', 'hml', 'umd']]
fitX_vwret = sm.add_constant(fitX_vwret)
rgmodel = sm.OLS(fitY_vwret, fitX_vwret).fit()
FFMom_alphas = pd.DataFrame(rgmodel.params.loc['const']).T * 100
FFMom_alphas.index = ['FFMom_alphas']; FFMom_alphas.columns = labelc
value_weighted_table = pd.concat([value_weighted_table, FFMom_alphas], axis=0)

# FF + Momentum + PSLiquility
fitXY_ewret = Y_ewret.merge(FF_dataset[['date', 'mktrf', 'smb', 'hml', 'umd', 'rf']], how='left', on='date')
fitXY_ewret = fitXY_ewret.merge(PSLiq_dataset, how='left', on='date')
fitY_ewret = fitXY_ewret[labelc] - mat(fitXY_ewret['rf']).T @ np.ones([1, 5])
fitX_ewret = fitXY_ewret[['mktrf', 'smb', 'hml', 'umd', 'PS_VWF']]
fitX_ewret = sm.add_constant(fitX_ewret)
rgmodel = sm.OLS(fitY_ewret, fitX_ewret).fit()
FFMomLiq_alphas = pd.DataFrame(rgmodel.params.loc['const']).T * 100
FFMomLiq_alphas.index = ['FFMomLiq_alphas']; FFMomLiq_alphas.columns = labelc
equal_weighted_table = pd.concat([equal_weighted_table, FFMomLiq_alphas], axis=0)

fitXY_vwret = Y_vwret.merge(FF_dataset[['date', 'mktrf', 'smb', 'hml', 'umd', 'rf']], how='left', on='date')
fitXY_vwret = fitXY_vwret.merge(PSLiq_dataset, how='left', on='date')
fitY_vwret = fitXY_vwret[labelc] - mat(fitXY_vwret['rf']).T @ np.ones([1, 5])
fitX_vwret = fitXY_vwret[['mktrf', 'smb', 'hml', 'umd', 'PS_VWF']]
fitX_vwret = sm.add_constant(fitX_vwret)
rgmodel = sm.OLS(fitY_vwret, fitX_vwret).fit()
FFMomLiq_alphas = pd.DataFrame(rgmodel.params.loc['const']).T * 100
FFMomLiq_alphas.index = ['FFMomLiq_alphas']; FFMomLiq_alphas.columns = labelc
value_weighted_table = pd.concat([value_weighted_table, FFMomLiq_alphas], axis=0)

equal_weighted_table['High-Low'] = equal_weighted_table['High'] - equal_weighted_table['Low']
equal_weighted_table = equal_weighted_table.round(2)
value_weighted_table['High-Low'] = value_weighted_table['High'] - value_weighted_table['Low']
value_weighted_table = value_weighted_table.round(2)
# save
writer = pd.ExcelWriter(os.path.join(args.save_dir, f'PortfolioSort_HoldingPeriod_{args.H}M_{args.dataset_start}_{args.dataset_end}_loading_{args.loading}.xlsx'))
equal_weighted_table.to_excel(writer, sheet_name='ewretd')
value_weighted_table.to_excel(writer, sheet_name='vwretd')
writer.save()
writer.close()
print(equal_weighted_table)
print(value_weighted_table)


                        Low     2     3     4  High  High-Low
Equal weighted AvgRET  0.95  1.13  1.21  1.28  1.22      0.27
CAPM_alpha            -0.07  0.23  0.29  0.32  0.18      0.25
FF_alphs              -0.16  0.07  0.12  0.15  0.05      0.21
FFMom_alphas          -0.12  0.08  0.15  0.19  0.13      0.25
FFMomLiq_alphas       -0.13  0.08  0.16  0.20  0.15      0.27
                        Low     2     3     4  High  High-Low
Value weighted AvgRET  0.81  0.98  0.99  1.11  1.09      0.28
CAPM_alpha            -0.15  0.10  0.08  0.12  0.01      0.16
FF_alphs              -0.11  0.07  0.05  0.09  0.02      0.13
FFMom_alphas          -0.09  0.02  0.04  0.11  0.13      0.21
FFMomLiq_alphas       -0.12  0.02  0.05  0.13  0.15      0.27
